In [1]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
import numpy as np
import cv2
import os

In [2]:
prototxtPath=os.path.sep.join([r'C:\Users\fathi\Desktop\project','deploy.prototxt'])
weightsPath=os.path.sep.join([r'C:\Users\fathi\Desktop\project','res10_300x300_ssd_iter_140000.caffemodel'])


In [3]:
prototxtPath

'C:\\Users\\fathi\\Desktop\\project\\deploy.prototxt'

In [4]:
weightsPath

'C:\\Users\\fathi\\Desktop\\project\\res10_300x300_ssd_iter_140000.caffemodel'

In [5]:
net=cv2.dnn.readNet(prototxtPath,weightsPath)

In [6]:
model=load_model(r'C:\Users\fathi\Desktop\project\mobilenet_v2.model')

In [7]:
image=cv2.imread(r'C:\Users\fathi\Desktop\project\WhatsApp Image 2021-10-30 at 3.41.05 PM.jpeg')

In [8]:
image

array([[[185, 175, 165],
        [184, 174, 164],
        [184, 174, 164],
        ...,
        [217, 219, 219],
        [216, 218, 218],
        [216, 218, 218]],

       [[185, 175, 165],
        [184, 174, 164],
        [184, 174, 164],
        ...,
        [217, 219, 219],
        [216, 218, 218],
        [216, 218, 218]],

       [[185, 175, 165],
        [184, 174, 164],
        [184, 174, 164],
        ...,
        [217, 219, 219],
        [216, 218, 218],
        [216, 218, 218]],

       ...,

       [[ 88, 131, 134],
        [ 86, 129, 132],
        [ 79, 127, 129],
        ...,
        [ 62, 137, 139],
        [ 60, 135, 137],
        [ 59, 134, 136]],

       [[101, 141, 146],
        [ 96, 138, 143],
        [ 90, 135, 139],
        ...,
        [ 62, 137, 139],
        [ 60, 135, 137],
        [ 59, 134, 136]],

       [[107, 147, 152],
        [104, 144, 149],
        [ 97, 139, 144],
        ...,
        [ 62, 137, 139],
        [ 60, 135, 137],
        [ 59, 134, 136]]

In [9]:
(h,w)=image.shape[:2]

In [10]:
(h,w)

(1280, 960)

In [11]:
blob=cv2.dnn.blobFromImage(image,1.0,(300,300),(104.0,177.0,123.0))

In [12]:
blob

array([[[[ 80.,  79.,  78., ..., 117., 115., 112.],
         [ 85.,  83.,  82., ..., 116., 115., 112.],
         [ 94.,  93.,  92., ..., 118., 117., 116.],
         ...,
         [-64., -60., -56., ..., -54., -48., -47.],
         [-68., -66., -64., ..., -57., -50., -51.],
         [-15., -36., -52., ..., -53., -43., -44.]],

        [[ -3.,  -4.,  -4., ...,  46.,  44.,  41.],
         [  3.,   1.,   0., ...,  45.,  44.,  41.],
         [ 12.,  11.,  10., ...,  47.,  46.,  45.],
         ...,
         [-72., -66., -62., ..., -49., -43., -43.],
         [-85., -77., -70., ..., -52., -46., -47.],
         [-45., -56., -62., ..., -50., -41., -42.]],

        [[ 41.,  40.,  39., ..., 100.,  98.,  95.],
         [ 47.,  45.,  44., ...,  99.,  98.,  95.],
         [ 57.,  56.,  55., ..., 101., 100.,  99.],
         ...,
         [-17., -10.,  -5., ...,   4.,  12.,  13.],
         [-30., -23., -16., ...,   0.,   9.,   9.],
         [ 13.,   1.,  -5., ...,   3.,  15.,  14.]]]], dtype=float32)

In [13]:
blob.shape

(1, 3, 300, 300)

In [14]:
net.setInput(blob)
detections=net.forward()

In [15]:
detections

array([[[[ 0.        ,  1.        ,  0.99985766,  0.44929814,
           0.09679946,  0.6238562 ,  0.282634  ],
         [ 0.        ,  1.        ,  0.12676378,  0.02410614,
           0.12805003,  0.99238443,  0.8790942 ],
         [ 0.        ,  1.        ,  0.12515053,  4.156394  ,
           4.002614  ,  4.84074   ,  4.9817934 ],
         [ 0.        ,  1.        ,  0.1209005 ,  4.013297  ,
           0.13056865,  4.972925  ,  0.8788283 ],
         [ 0.        ,  1.        ,  0.11499635,  0.15608454,
           4.0084167 ,  0.8431396 ,  4.9816413 ],
         [ 0.        ,  1.        ,  0.09732129,  0.84982044,
           0.16432512,  0.9994535 ,  0.39309436],
         [ 0.        ,  1.        ,  0.09702609,  0.5326462 ,
           0.507055  ,  0.5621147 ,  0.5544643 ],
         [ 0.        ,  1.        ,  0.09616662,  0.48562533,
           0.49573904,  0.5179216 ,  0.5421997 ],
         [ 0.        ,  1.        ,  0.09552404,  2.032616  ,
           0.1312786 ,  2.9554484 ,  0.880

In [20]:
#loop over the detections
for i in range(0,detections.shape[2]):
    confidence=detections[0,0,i,2]
    
    
    if confidence>0.5:
        #we need the X,Y coordinates
        box=detections[0,0,i,3:7]*np.array([w,h,w,h])
        (startX,startY,endX,endY)=box.astype('int')
        
        #ensure the bounding boxes fall within the dimensions of the frame
        (startX,startY)=(max(0,startX),max(0,startY))
        (endX,endY)=(min(w-1,endX), min(h-1,endY))
        
        
        #extract the face ROI, convert it from BGR to RGB channel, resize it to 224,224 and preprocess it
        face=image[startY:endY, startX:endX]
        face=cv2.cvtColor(face,cv2.COLOR_BGR2RGB)
        face=cv2.resize(face,(224,224))
        face=img_to_array(face)
        face=preprocess_input(face)
        face=np.expand_dims(face,axis=0)
        
        (mask,withoutMask)=model.predict(face)[0]
        
        #determine the class label and color we will use to draw the bounding box and text
        label='Mask' if mask>withoutMask else 'No Mask'
        color=(0,255,0) if label=='Mask' else (0,0,255)
        
        #include the probability in the label
        label="{}: {:.2f}%".format(label,max(mask,withoutMask)*100)
        
        #display the label and bounding boxes
        cv2.putText(image,label,(startX,startY-10),cv2.FONT_HERSHEY_SIMPLEX,0.45,color,2)
        cv2.rectangle(image,(startX,startY),(endX,endY),color,2)
        
        
        
cv2.imshow("OutPut",image)
cv2.waitKey(0)
cv2.destroyAllWindows()